In [31]:
import pyforest
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer

In [32]:
import os
files_and_directories = os.listdir("data/")
all_files = []
for file in files_and_directories:
    if 'csv' in file:
        temp = pd.read_csv('data/'+str(file))
        all_files.append(temp)
        
df = pd.concat(all_files, sort=False)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [33]:
df.tail()

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,SJA,BSH,BSD,BSA,SBH,SBD,SBA,0,1,2
375,E0,2010-05-09,Everton,Portsmouth,1.0,0.0,H,0.0,0.0,D,...,11.00,1.30,5.00,10.00,1.29,5.00,8.50,NaN,NaN,NaN
376,E0,2010-05-09,Hull,Liverpool,0.0,0.0,D,0.0,0.0,D,...,1.57,6.00,3.75,1.57,6.00,3.75,1.50,NaN,NaN,NaN
377,E0,2010-05-09,Man United,Stoke,4.0,0.0,H,2.0,0.0,H,...,21.00,1.11,7.50,23.00,1.11,7.50,15.00,NaN,NaN,NaN
378,E0,2010-05-09,West Ham,Man City,1.0,1.0,D,1.0,1.0,D,...,1.91,4.20,3.60,1.80,4.00,3.40,1.80,NaN,NaN,NaN
379,E0,2010-05-09,Wolves,Sunderland,2.0,1.0,H,1.0,1.0,D,...,3.00,2.38,3.25,3.00,2.25,3.20,2.88,NaN,NaN,NaN


In [34]:
df['Date'] = pd.to_datetime(df['Date'])
df.dropna(axis=1, how='all', inplace=True)
print(df.shape)

<IPython.core.display.Javascript object>

(7610, 80)


In [35]:
missing_columns = pd.DataFrame(df.isnull().sum())
missing_columns.columns = ['Cnt']
missing_columns.head()

<IPython.core.display.Javascript object>

,Cnt
Div,10
Date,10
HomeTeam,10
AwayTeam,10
FTHG,10


In [36]:
for idx, r in missing_columns.iterrows():
    if r['Cnt']>30:
        df.drop(idx,axis=1,inplace=True)
        
df.shape

(7610, 56)

In [37]:
# transform OHE the FTR (full time 1/N/2) and the Half Time (1/N/2)
df.reset_index(inplace=True, drop=True)
temp_ohe = pd.get_dummies(df[['FTR','HTR']])
df = df.merge(temp_ohe, left_index=True, right_index=True)
df.drop(['Div'], axis=1, inplace=True)

<IPython.core.display.Javascript object>

In [41]:
# encode the refereee
imp = SimpleImputer(strategy="most_frequent")
referee = imp.fit_transform(df[['Referee']])
df['Referee']=referee
oe = OrdinalEncoder()
referee = oe.fit_transform(df[['Referee']])

df['Referee']=referee

df.shape

(7610, 61)

In [47]:
df.dropna(axis=0, inplace=True)

In [50]:
df.isnull().sum().sum()

0

In [52]:
df.shape

(7576, 61)

In [53]:
df.columns

Index(['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG',
       'HTR', 'Referee', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC',
       'HY', 'AY', 'HR', 'AR', 'B365H', 'B365D', 'B365A', 'BWH', 'BWD', 'BWA',
       'IWH', 'IWD', 'IWA', 'WHH', 'WHD', 'WHA', 'VCH', 'VCD', 'VCA', 'Bb1X2',
       'BbMxH', 'BbAvH', 'BbMxD', 'BbAvD', 'BbMxA', 'BbAvA', 'BbOU',
       'BbMx>2.5', 'BbAv>2.5', 'BbMx<2.5', 'BbAv<2.5', 'BbAH', 'BbAHh',
       'BbMxAHH', 'BbAvAHH', 'BbMxAHA', 'BbAvAHA', 'FTR_A', 'FTR_D', 'FTR_H',
       'HTR_A', 'HTR_D', 'HTR_H'],
      dtype='object')

# Data definition
http://www.football-data.co.uk/notes.txt

### Key to results data:

- Div = League Division
- Date = Match Date (dd/mm/yy)
- Time = Time of match kick off
- HomeTeam = Home Team
- AwayTeam = Away Team
- FTHG and HG = Full Time Home Team Goals
- FTAG and AG = Full Time Away Team Goals
- FTR and Res = Full Time Result (H=Home Win, D=Draw, A=Away Win)
- HTHG = Half Time Home Team Goals
- HTAG = Half Time Away Team Goals
- HTR = Half Time Result (H=Home Win, D=Draw, A=Away Win)

### Match Statistics (where available)
- Attendance = Crowd Attendance
- Referee = Match Referee
- HS = Home Team Shots
- AS = Away Team Shots
- HST = Home Team Shots on Target
- AST = Away Team Shots on Target
- HHW = Home Team Hit Woodwork
- AHW = Away Team Hit Woodwork
- HC = Home Team Corners
- AC = Away Team Corners
- HF = Home Team Fouls Committed
- AF = Away Team Fouls Committed
- HFKC = Home Team Free Kicks Conceded
- AFKC = Away Team Free Kicks Conceded
- HO = Home Team Offsides
- AO = Away Team Offsides
- HY = Home Team Yellow Cards
- AY = Away Team Yellow Cards
- HR = Home Team Red Cards
- AR = Away Team Red Cards
- HBP = Home Team Bookings Points (10 = yellow, 25 = red)
- ABP = Away Team Bookings Points (10 = yellow, 25 = red)

### Key to 1X2 (match) betting odds data:

- B365H = Bet365 home win odds
- B365D = Bet365 draw odds
- B365A = Bet365 away win odds
- BSH = Blue Square home win odds
- BSD = Blue Square draw odds
- BSA = Blue Square away win odds
- BWH = Bet&Win home win odds
- BWD = Bet&Win draw odds
- BWA = Bet&Win away win odds
- GBH = Gamebookers home win odds
- GBD = Gamebookers draw odds
- GBA = Gamebookers away win odds
- IWH = Interwetten home win odds
- IWD = Interwetten draw odds
- IWA = Interwetten away win odds
- LBH = Ladbrokes home win odds
- LBD = Ladbrokes draw odds
- LBA = Ladbrokes away win odds
- PSH and PH = Pinnacle home win odds
- PSD and PD = Pinnacle draw odds
- PSA and PA = Pinnacle away win odds
- SOH = Sporting Odds home win odds
- SOD = Sporting Odds draw odds
- SOA = Sporting Odds away win odds
- SBH = Sportingbet home win odds
- SBD = Sportingbet draw odds
- SBA = Sportingbet away win odds
- SJH = Stan James home win odds
- SJD = Stan James draw odds
- SJA = Stan James away win odds
- SYH = Stanleybet home win odds
- SYD = Stanleybet draw odds
- SYA = Stanleybet away win odds
- VCH = VC Bet home win odds
- VCD = VC Bet draw odds
- VCA = VC Bet away win odds
- WHH = William Hill home win odds
- WHD = William Hill draw odds
- WHA = William Hill away win odds

## Explore the betting columns

The target is to build set of pronostics [x.yy, x.yy, x.yy] which means Home, D, Away for each betting platform.

Missing value will be filled with the line (other betting platform).

Betting score are always in 3 features which ended by:
- "H" for home
- "D" for deuce
- "A" for Away

### Extract columns from the dataset

Extract columns from dataset and build a dictionary of feature name by betting platform


In [54]:
all_betting_cols = [x for x in df.columns.tolist() if str(x).endswith("H") 
                    or str(x).endswith("D") or str(x).endswith("A")]
all_betting_cols

# from that list extract the betting platform
betting_platform={}
for v in all_betting_cols:
    bp = v[:-1]
    if bp != 'FTR_' and bp != 'HTR_':
        betting_platform[bp] = [f"{bp}H",f"{bp}D",f"{bp}A"]

betting_platform

{'B365': ['B365H', 'B365D', 'B365A'],
 'BW': ['BWH', 'BWD', 'BWA'],
 'IW': ['IWH', 'IWD', 'IWA'],
 'WH': ['WHH', 'WHD', 'WHA'],
 'VC': ['VCH', 'VCD', 'VCA'],
 'BbMx': ['BbMxH', 'BbMxD', 'BbMxA'],
 'BbAv': ['BbAvH', 'BbAvD', 'BbAvA'],
 'BbA': ['BbAH', 'BbAD', 'BbAA'],
 'BbMxAH': ['BbMxAHH', 'BbMxAHD', 'BbMxAHA'],
 'BbAvAH': ['BbAvAHH', 'BbAvAHD', 'BbAvAHA']}

### create the new dataset
read each row of the original dataset and for the betting platform create a new row, the dictionary will be
- day features such as teams and score
- date
- betting platform odds


In [64]:
new_set = []
for idx, r in df.iterrows():
    static_variable = [r.HomeTeam, r.AwayTeam]
    for key in betting_platform:
        temp = betting_platform[key]
        temp_odds =r[temp]
        new_set.append([static_variable[0],
                        static_variable[1],
                        temp_odds[0], temp_odds[1], temp_odds[2],
                        r.FTR]
                        )


In [65]:
bet=pd.DataFrame(new_set)
bet.dropna(inplace=True)
bet.shape

<IPython.core.display.Javascript object>

(53032, 6)

In [66]:
bet.isnull().sum()

0    0
1    0
2    0
3    0
4    0
5    0
dtype: int64

In [67]:
bet.dtypes

0     object
1     object
2    float64
3    float64
4    float64
5     object
dtype: object

In [68]:
bet.sample(6)

,0,1,2,3,4,5
6913,Liverpool,West Ham,1.36,5.0,8.00,D
69243,Swansea,Stoke,1.95,3.4,4.00,D
22761,Tottenham,Birmingham,1.50,4.0,6.00,H
8162,Sunderland,Wigan,1.90,3.3,3.70,H
16944,Huddersfield,Southampton,3.10,3.1,2.63,A
57294,Everton,Wigan,1.53,4.0,8.00,H


In [ ]:
#bet.loc[:,5] = bet.loc[:,5].map({'H':-1,'D':0,'A':1})
bet.sample(9)

In [ ]:
oe = OrdinalEncoder()
bet['H']=oe.fit_transform(np.array(bet.loc[:,0].tolist()).reshape(-1,1))
bet['A']=oe.fit_transform(np.array(bet.loc[:,1].tolist()).reshape(-1,1))

In [ ]:
bet.columns=['Hteam','Ateam','1','N','2','Result','TeamH','TeamA']
bet.head()

In [ ]:
bet.to_csv('bet.csv', index=False)